<a href="https://colab.research.google.com/github/rajeevradnair/LLM_Engineering/blob/main/company_brochure_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai

In [ ]:
pip install iPython

In [ ]:
pip install ipython

In [96]:
from google.colab import userdata
from openai import OpenAI

#fetch API key to OpenAI API layer
openai_api_key=userdata.get('OPENAI_API_KEY')
openai=OpenAI(api_key=openai_api_key)

#create an OpenAI client
if openai:
  print(f'openai client created: {openai} ')

else:
  print(f'Issues in creating a client to the OpenAI API layer')

openai client created: <openai.OpenAI object at 0x7cd0b65583d0> 


In [ ]:
from typing import List
from bs4 import BeautifulSoup
import requests

#Website parser, collects title, text, body & links on the page at a given URL
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
  def __init__(self, url) -> None:
    self.url=url
    response=requests.get(url, headers=headers)
    self.body=response.content
    soup=BeautifulSoup(self.body, 'html.parser')
    self.title=soup.title.string if soup.title else 'No title found'
    if soup.body:
      for irrelevant in soup.body(["script", "style", "img", "input"]):
        irrelevant.decompose()
      self.text = soup.body.get_text(separator="\n", strip=True)
    else:
        self.text = ""
    links = [link.get('href') for link in soup.find_all('a')]

    self.links = [link for link in links if link]

  def get_contents(self):
    return f'Webpage Title:\n{self.title}\nWebPage Contents:\n{self.text}'


ws = Website("https://apple.com")
ws.links

In [ ]:
#Extract relevant links given a primary link
THEME=['','SALES', 'MARKETING', 'VISION', 'SENIOR MANAGEMENT']

link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

def get_links_user_prompt(website, theme=''):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for"
    if theme:
      user_prompt += f" the {theme} portion of"
    user_prompt += " the brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

print(get_links_user_prompt(ws,THEME[1]))


In [100]:
#Extract relevant links given a primary link
MODEL = 'gpt-4o-mini'

import json
def get_links(url, theme):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website, theme)}
        ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

get_links('https://google.com')

{'links': [{'type': 'about page',
   'url': 'https://about.google/?fg=1&utm_source=google-US&utm_medium=referral&utm_campaign=hp-header'},
  {'type': 'store page',
   'url': 'https://store.google.com/US?utm_source=hp_header&utm_medium=google_ooo&utm_campaign=GS100042&hl=en-US'},
  {'type': 'ads page',
   'url': 'https://www.google.com/intl/en_us/ads/?subid=ww-ww-et-g-awa-a-g_hpafoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpafooter&fg=1'},
  {'type': 'services page',
   'url': 'https://www.google.com/services/?subid=ww-ww-et-g-awa-a-g_hpbfoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpbfooter&fg=1'}]}

In [ ]:
#Extract all URLs from given a primary URL, find relevant ones and extract details of the relevant URLs
def get_all_details(url, theme):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url, theme)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

print(get_all_details("https://google.com"))


In [ ]:
#Systems and user prompts to create a brochure given the details of all extracted, but relevant URLs
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure in English about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

get_brochure_user_prompt("Google", "https://google.com")


In [ ]:
#Final output i.e. the Brochure
from IPython.display import Markdown, display, update_display
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

create_brochure("Google", "https://google.com")


In [107]:
#Final output i.e. the Brochure in streaming format
from IPython.display import Markdown, display, update_display
def create_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    response=''
    display_handle=display(Markdown(''), display_id=True)
    for chunk in stream:
      response+=chunk.choices[0].delta.content or ''
      response=response.replace("```","").replace("markdown", "")
      update_display(Markdown(response), display_id=display_handle.display_id)

create_brochure("Google", "https://google.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://about.google/?fg=1&utm_source=google-US&utm_medium=referral&utm_campaign=hp-header'}, {'type': 'store page', 'url': 'https://store.google.com/US?utm_source=hp_header&utm_medium=google_ooo&utm_campaign=GS100042&hl=en-US'}, {'type': 'ads page', 'url': 'https://www.google.com/intl/en_us/ads/?subid=ww-ww-et-g-awa-a-g_hpafoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpafooter&fg=1'}, {'type': 'services page', 'url': 'https://www.google.com/services/?subid=ww-ww-et-g-awa-a-g_hpbfoot1_1!o2&utm_source=google.com&utm_medium=referral&utm_campaign=google_hpbfooter&fg=1'}]}


# Google Company Brochure

## About Us
At Google, we strive to organize the world's information and make it universally accessible and useful. Founded in 1998, we have become a leader in technology, heavily investing in areas like artificial intelligence (AI) and cloud computing to enhance user experiences across our products.

## Our Products
Google offers a wide array of products and services, including:

- **Search Engine**: The most used search engine globally.
- **Google Workspace**: Tools like Gmail, Google Drive, Google Docs, and Google Meet for productivity and collaboration.
- **Hardware**: Innovative devices such as the Pixel smartphones, Pixel Buds, Google Nest products, and more, designed to integrate seamlessly with our services.

### AI at the Core
We're committed to integrating AI in meaningful ways, from enhancing search functionalities to developing advanced models that solve real-world challenges.

## Company Culture
At Google, we believe that our employees are our greatest asset. Our culture is centered around:

- **Innovation**: Encouraging teams to embrace creativity and back new ideas.
- **Diversity and Inclusion**: Fostering a workplace that celebrates diverse perspectives.
- **Well-being**: Prioritizing employee health through supportive programs and flexible working environments.

Our offices worldwide reflect this ethos, providing collaborative spaces and numerous amenities to ensure a fulfilling work-life balance.

## Customer Base
Our users span across individual consumers, small businesses, and large enterprises. Google is committed to providing tools that help users achieve more, whether it's enhancing personal productivity or powering complex enterprise solutions.

## Careers at Google
Google is a place where you can make an impact. We are always looking for talent who are passionate about technology and want to challenge the status quo. We offer:

- **Professional Growth**: Opportunities for learning and advancement in numerous fields.
- **Inclusive Environment**: A workforce that celebrates diversity and inclusivity.
- **Competitive Benefits**: Comprehensive health plans, retirement options, and unique perks to support employee well-being.

Explore your future with us by checking out our careers page!

## Join Us
Whether you're a prospective klant, investor, or talent, we welcome you to be part of our journey in shaping the future of technology. For inquiries or more information, visit our website!

---

**Contact Us**
For questions, support, or more information, visit our website at [Google.com](https://www.google.com).

**Follow Us**
Stay updated with the latest announcements and trends by subscribing to our news updates!